In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 204
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#252.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
using System.Collections;
using ilPSP.Connectors.Matlab;
using ilPSP.LinSolvers;
using BoSSS.Solution.Tecplot;

In [ ]:
static Func<double[], bool> IsWallBaseNoSlipBndy = null;
static Func<double[], bool> IsWallBaseSlipBndy = null;
static Func<double[], double[]> UWallBase = null;
static double lSlip = 0.1;

static Func<double[], bool> IsWallCornerNoSlipBndy = null;    // is true for wallBC
static Func<double[], bool> IsWallCornerSlipBndy = null;    // is true for slipBC
//Func<double[], double[]> UWallCorner = null;

static Func<double[], bool> IsOutletBndy = null;
static Func<double[], double[]> pOut = null;
static Func<double[], bool> IsVelOutletBndy = null;
static Func<double[], double[]> Uout = null;

static CellMask singlrtyMask;

In [ ]:
public class Divergence : BoSSS.Foundation.IEdgeForm, BoSSS.Foundation.IVolumeForm {

    bool localAnsatz;
    bool NoBndy;
    
    public Divergence(bool _locAnsatz = false, bool _NoBndy = false) {
         localAnsatz = _locAnsatz;
         NoBndy      = _NoBndy;
    }
    
    
    public IList<string> ParameterOrdering {   
        get { return null; }
    }
    
    public IList<string> ArgumentOrdering {   
        get { return new string[] { "u", "v" }; }
    }
    
    public TermActivationFlags VolTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public TermActivationFlags InnerEdgeTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public TermActivationFlags BoundaryEdgeTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public double VolumeForm(ref CommonParamsVol cpv, double[] U, double[,] GradU, double V, double[] GradV) {
        double Acc = 0;
        for(int d = 0; d < cpv.D; d++) {     
            Acc -= GradU[d,d]*V; 
        }
        return (!localAnsatz) ? Acc : 0.0;
    }
    
    public double InnerEdgeForm(ref CommonParams inp, double[] U_IN, double[] U_OT, double[,] GradU_IN, double[,] GradU_OT,
        double V_IN, double V_OT, double[] GradV_IN, double[] GradV_OT) {
    
        double Acc = 0;
        for(int d = 0; d < inp.D; d++) {
            Acc += 0.5*(V_IN + V_OT)*(U_IN[d] - U_OT[d])*inp.Normal[d];
        }      
        return (!localAnsatz) ? Acc : 0.0;
    }
    
    public double BoundaryEdgeForm(ref CommonParamsBnd inp, double[] U_IN, double[,] GradU_IN, double V_IN, double[] GradV_IN) {
    
        if(NoBndy)
            return 0.0;
        
        double Acc = 0.0; 

        // wall base
        if((IsWallBaseNoSlipBndy(inp.X))||(IsWallBaseSlipBndy(inp.X))){
            double[] UD = UWallBase(inp.X);
            for(int d = 0; d < inp.D; d++) {          
                Acc += (U_IN[d] - UD[d])*inp.Normal[d]*V_IN;
            }
        }
        // wall corner
        if((IsWallCornerNoSlipBndy(inp.X))||(IsWallCornerSlipBndy(inp.X))){
            for(int d = 0; d < inp.D; d++) {          
                Acc += (U_IN[d] - 0.0)*inp.Normal[d]*V_IN;
            }
        }
        // outlet
        if(IsOutletBndy(inp.X)){
        // Acc = 0.0;
        }

        return (!localAnsatz) ? Acc : 0.0;
    }

}

In [ ]:
public class Gradient_d : BoSSS.Foundation.IEdgeForm, BoSSS.Foundation.IVolumeForm {

    bool NoBndy;
    
    public Gradient_d(int _d, bool _NoBndy = false) {
        this.d = _d;
        NoBndy = _NoBndy;
    }
    
    int d;
    
    public IList<string> ParameterOrdering {   
        get { return null; }
    }
    
    public IList<string> ArgumentOrdering {   
        get { return new string[] { "p" }; }
    }
    
    public TermActivationFlags VolTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public TermActivationFlags InnerEdgeTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public TermActivationFlags BoundaryEdgeTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public double VolumeForm(ref CommonParamsVol cpv, double[] U, double[,] GradU, double V, double[] GradV) {
        double Acc = 0;  
        Acc -= U[0]*GradV[d]; 
        return Acc;
    }
    
    public double InnerEdgeForm(ref CommonParams inp, double[] U_IN, double[] U_OT, double[,] GradU_IN, double[,] GradU_OT,
        double V_IN, double V_OT, double[] GradV_IN, double[] GradV_OT) {
    
        double Acc = 0;
        Acc += 0.5*(U_IN[0] + U_OT[0])*inp.Normal[d]*(V_IN - V_OT); 
        return Acc;
    }
    
    public double BoundaryEdgeForm(ref CommonParamsBnd inp, double[] U_IN, double[,] GradU_IN, double V_IN, double[] GradV_IN) {
    
        if(NoBndy)
            return 0.0;
        
        double Acc = 0.0;

        if(IsOutletBndy(inp.X)){
            Acc = pOut(inp.X)[0]*inp.Normal[d]*V_IN;
        } else {
            Acc = U_IN[0]*inp.Normal[d]*V_IN;  
        }

        return Acc;
    }

}

In [ ]:
public class LogGradient_d : BoSSS.Foundation.IEdgeForm, BoSSS.Foundation.IVolumeForm {

    bool NoBndy;
    
    public LogGradient_d(int _d, bool _NoBndy = false) {
        this.d = _d;
        NoBndy = _NoBndy;
    }
    
    int d;
    
    public IList<string> ParameterOrdering {   
        get { return null; }
    }
    
    public IList<string> ArgumentOrdering {   
        get { return new string[] { "p" }; }
    }
    
    public TermActivationFlags VolTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public TermActivationFlags InnerEdgeTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public TermActivationFlags BoundaryEdgeTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    private double GetDistanceToSingularity(double[] X){
        return Math.Sqrt(X[0].Pow2() + X[1].Pow2());
    }
    
    public double VolumeForm(ref CommonParamsVol cpv, double[] U, double[,] GradU, double V, double[] GradV) {
        double Acc = 0;  
        double r   = GetDistanceToSingularity(cpv.Xglobal);
        Acc -= (1.0/lSlip)*Math.Log(r)*GradV[d]; 
        return (singlrtyMask.Contains(cpv.jCell)) ? Acc : 0.0;
    }
    
    public double InnerEdgeForm(ref CommonParams inp, double[] U_IN, double[] U_OT, double[,] GradU_IN, double[,] GradU_OT,
        double V_IN, double V_OT, double[] GradV_IN, double[] GradV_OT) {
    
        double Acc = 0;
        double r   = GetDistanceToSingularity(inp.X);
        Acc += (1.0/lSlip)*Math.Log(r)*inp.Normal[d]*(V_IN - V_OT); 
        return (singlrtyMask.Contains(inp.jCellIn)) ? Acc : 0.0;
    }
    
    public double BoundaryEdgeForm(ref CommonParamsBnd inp, double[] U_IN, double[,] GradU_IN, double V_IN, double[] GradV_IN) {
    
        if(NoBndy)
            return 0.0;
        
        double Acc = 0.0;

        if(IsOutletBndy(inp.X)){
            Acc = pOut(inp.X)[0]*inp.Normal[d]*V_IN;
        } else {
            double r = GetDistanceToSingularity(inp.X);
            Acc      = (1.0/lSlip)*Math.Log(r)*inp.Normal[d]*V_IN;  
        }
        
        //double r = GetDistanceToSingularity(inp.X);
        //Acc      = Math.Log(r)*inp.Normale[d]*V_IN;  

        return (singlrtyMask.Contains(inp.jCellIn)) ? Acc : 0.0;
    }

}

In [ ]:
public class ConstGradient_d : BoSSS.Foundation.IEdgeForm, BoSSS.Foundation.IVolumeForm {

    bool NoBndy;
    
    public ConstGradient_d(int _d, bool _NoBndy = false) {
        this.d = _d;
        NoBndy = _NoBndy;
    }
    
    int d;
    
    public IList<string> ParameterOrdering {   
        get { return null; }
    }
    
    public IList<string> ArgumentOrdering {   
        get { return new string[] { "p" }; }
    }
    
    public TermActivationFlags VolTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public TermActivationFlags InnerEdgeTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public TermActivationFlags BoundaryEdgeTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    
    public double VolumeForm(ref CommonParamsVol cpv, double[] U, double[,] GradU, double V, double[] GradV) {
        double Acc = 0;  
        Acc -= 1.0*GradV[d]; 
        return (singlrtyMask.Contains(cpv.jCell)) ? Acc : 0.0;
    }
    
    public double InnerEdgeForm(ref CommonParams inp, double[] U_IN, double[] U_OT, double[,] GradU_IN, double[,] GradU_OT,
        double V_IN, double V_OT, double[] GradV_IN, double[] GradV_OT) {
    
        double Acc = 0;
        Acc += 1.0*inp.Normal[d]*(V_IN - V_OT); 
        return (singlrtyMask.Contains(inp.jCellIn)) ? Acc : 0.0;
    }
    
    public double BoundaryEdgeForm(ref CommonParamsBnd inp, double[] U_IN, double[,] GradU_IN, double V_IN, double[] GradV_IN) {
    
        if(NoBndy)
            return 0.0;
        
        double Acc = 0.0;

        if(IsOutletBndy(inp.X)){
            Acc = pOut(inp.X)[0]*inp.Normal[d]*V_IN;
        } else {
            Acc = 1.0*inp.Normal[d]*V_IN;  
        }
        
        //Acc = 1.0*inp.Normale[d]*V_IN;  

        return (singlrtyMask.Contains(inp.jCellIn)) ? Acc : 0.0;
    }

}

In [ ]:
public class Viscous : BoSSS.Foundation.IEdgeForm, BoSSS.Foundation.IVolumeForm {

    bool localAnsatz;
    bool NoBndy;
    
    public Viscous(int _d, bool _locAnsatz = false, bool _NoBndy = false) {
        localAnsatz = _locAnsatz;
        NoBndy      = _NoBndy;
        this.m_d    = _d;
    }
    
    int m_d;
    
    double m_alpha = 1.0;
    
    public static double Re;
    public static int PolynomialDegree;
    public static MultidimensionalArray cj;
    
    public IList<string> ParameterOrdering {   
        get { return null; }
    }
    
    public IList<string> ArgumentOrdering {   
        get { return new string[] { "u", "v" }; }
    }
    
    public TermActivationFlags VolTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public TermActivationFlags InnerEdgeTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public TermActivationFlags BoundaryEdgeTerms {   
        get { return TermActivationFlags.AllOn;}
    }
    
    public double VolumeForm(ref CommonParamsVol cpv, double[] U, double[,] GradU, double V, double[] GradV) {
        double Acc = 0.0; 
        for(int d = 0; d < cpv.D; d++) {        
            Acc += GradU[m_d, d]*GradV[d]; 
        }
        return (!localAnsatz) ? (1/Re)*Acc : 0.0;
    }
    
    public double InnerEdgeForm(ref CommonParams inp, double[] U_IN, double[] U_OT, double[,] GradU_IN, double[,] GradU_OT,
        double V_IN, double V_OT, double[] GradV_IN, double[] GradV_OT) {
    
        double eta = PenaltyFactor(inp.jCellIn, inp.jCellOut);
        
        double Acc = 0.0;
        for(int d = 0; d < inp.D; d++) {    
            Acc -= 0.5*(GradU_IN[m_d, d] + GradU_OT[m_d, d])*(V_IN - V_OT)*inp.Normal[d]*m_alpha; 
            Acc -= 0.5*(GradV_IN[d] + GradV_OT[d])*(U_IN[m_d] - U_OT[m_d])*inp.Normal[d]*m_alpha; 
        }
        Acc += eta*(U_IN[m_d] - U_OT[m_d])*(V_IN - V_OT); 
        return (!localAnsatz) ? (1/Re)*Acc : 0.0;
    }
    
    public double BoundaryEdgeForm(ref CommonParamsBnd inp, double[] U_IN, double[,] GradU_IN, double V_IN, double[] GradV_IN) {
    
        if(NoBndy)
            return 0.0;
                
        double Acc = 0.0;
        double eta = PenaltyFactor(inp.jCellIn, -1);
               
        if(IsWallBaseNoSlipBndy(inp.X)||IsWallCornerNoSlipBndy(inp.X)) {      

            double uBnd;
            if(IsWallBaseNoSlipBndy(inp.X)) {
                uBnd = UWallBase(inp.X)[m_d];
            } else {
                uBnd = 0.0;
            }
            
            for(int d = 0; d < inp.D; d++) {    
                Acc -= GradU_IN[m_d, d]*V_IN*inp.Normal[d]*m_alpha; 
                Acc -= GradV_IN[d]*(U_IN[m_d] - uBnd)*inp.Normal[d]*m_alpha; 
            }

            Acc += eta*(U_IN[m_d] - uBnd)*V_IN; 
            
            return (!localAnsatz) ? (1/Re)*Acc : 0.0; 
        }        
        
        if(IsOutletBndy(inp.X)) {

            return 0.0;
        }
        
        if(IsWallBaseSlipBndy(inp.X)||IsWallCornerSlipBndy(inp.X)) {
        
            int D = inp.D;
            double[] g_D; // = UWallBase(inp.X);
            double beta; // = 1/lSlip;
            
            if(IsWallBaseSlipBndy(inp.X)){
                g_D  = UWallBase(inp.X);
                beta = 1.0/lSlip;
            } else {
                g_D  = new double[] {0.0, 0.0};
                beta = 0.0;
            }

            for(int dN = 0; dN < D; dN++) {
                for(int dD = 0; dD < D; dD++) {
                    // consistency
                    Acc -= (inp.Normal[dN] * GradU_IN[dN, dD] * inp.Normal[dD]) * (V_IN * inp.Normal[m_d]) * m_alpha;
                    // symmetry
                    Acc -= (inp.Normal[m_d] * GradV_IN[dD] * inp.Normal[dD]) * (U_IN[dN] - g_D[dN]) * inp.Normal[dN] * m_alpha;
                }
                // penalty
                Acc += ((U_IN[dN] - g_D[dN]) * inp.Normal[dN]) * ((V_IN - 0) * inp.Normal[m_d]) * eta;
            }
            
             double[,] P = new double[D, D];
             for(int d1 = 0; d1 < D; d1++) {
             for(int d2 = 0; d2 < D; d2++) {
                 double nn = inp.Normal[d1] * inp.Normal[d2];
                 if(d1 == d2) {
                     P[d1, d2] = 1 - nn;
                 } else {
                     P[d1, d2] = -nn;
                 }
             }
             }

             // tangential dissipation force term
             for(int d1 = 0; d1 < D; d1++) {
                 for(int d2 = 0; d2 < D; d2++) {
                     //g_D = USlip(inp.X)[d2];
                     Acc += (beta * P[d1, d2] * (U_IN[d2] - g_D[d2])) * (P[d1, m_d] * V_IN) * m_alpha;
                 }
             }
            
             return (!localAnsatz) ? (1/Re)*Acc : 0.0; 
        }

        
        return (!localAnsatz) ? (1/Re)*Acc : 0.0; 
    }
    
    
    double PenaltyFactor(int jCellIn, int jCellOut) {
        double PenaltySafety = 2;
        double cj_in         = cj[jCellIn];
        double penalty_base  = PenaltySafety * PolynomialDegree*PolynomialDegree; 
        double eta           = penalty_base * cj_in; 
        if(jCellOut >= 0) {
            double cj_out = cj[jCellOut];
            eta           = Math.Max(eta, penalty_base*cj_out);
        }
        return eta;
    }

};

In [ ]:
double baseL = 10;
double theta = Math.PI*1.0/4.0;
int lvl      = 3;
Grid2D grd_Corner;

In [ ]:
if (theta<(Math.PI/2)){

    grd_Corner = Grid2D.AcuteCornerTriangleGrid(baseL, theta, lvl);
    
    IsWallBaseSlipBndy   = (X => (Math.Abs(X[1]) <= 1.0e-8));
    IsWallBaseNoSlipBndy = (X => false);
    UWallBase            = (X => new double[] {-1.0, 0.0});
    
    IsWallCornerSlipBndy   = (X => (X[0] > -baseL * Math.Cos(theta)) && (X[1] > 1.0e-8));
    IsWallCornerNoSlipBndy = (X => false); //(X[0] > -baseL * Math.Cos(theta)) && (X[1] > 1.0e-8));

    IsOutletBndy = (X =>(X[0] < -baseL * Math.Cos(theta)) && (X[1] > 1.0e-8));
    pOut         = (X => new double[] {0.0});
    
} else {

    grd_Corner = Grid2D.ObtuseCornerTriangleGrid(baseL, theta, lvl);

    IsWallBaseSlipBndy   = (X => (Math.Abs(X[1]) <= 1.0e-8));
    IsWallBaseNoSlipBndy = (X => false);
    UWallBase            = (X => new double[] {-1.0, 0.0});
    
    IsWallCornerSlipBndy   = (X => (X[0] > 1.0e-8) && (X[1] < baseL * Math.Cos(theta - (Math.PI/2.0))));
    IsWallCornerNoSlipBndy = (X => false); //(X[0] > 1.0e-8) && (X[1] < baseL * Math.Cos(theta - (Math.PI/2.0))));
    
    IsOutletBndy = (X => ((X[0] <= baseL * (Math.Sin(theta - (Math.PI / 2.0)) - 1)) && (X[1] > 1.0e-8)) || (Math.Abs(X[1] - baseL * Math.Cos(theta - (Math.PI / 2.0))) <= 1.0e-8));
    pOut         = (X => new double[] {0.0});
    
};
var grdDat_Corner = new GridData(grd_Corner);

In [ ]:
BitArray CellArray                = new BitArray(grdDat_Corner.Cells.NoOfLocalUpdatedCells);
MultidimensionalArray CellCenters = grdDat_Corner.Cells.CellCenter;
double singlrtyR                  = 10;
for(int i = 0; i < grdDat_Corner.Cells.NoOfLocalUpdatedCells; i++) {
    double r = Math.Sqrt(CellCenters[i,0].Pow2() + CellCenters[i,1].Pow2());
    CellArray[i] = (r < singlrtyR);
}
singlrtyMask = new CellMask(grdDat_Corner, CellArray, MaskType.Logical);

In [ ]:
singlrtyMask.NoOfItemsLocally_WithExternal

64

In [ ]:
var VelB_Corner   = new Basis(grdDat_Corner, 2); 
var PsiB_Corner   = new Basis(grdDat_Corner, 1); 
var varMap_Corner = new UnsetteledCoordinateMapping(VelB_Corner, VelB_Corner, PsiB_Corner); 

In [ ]:
int locOrder = 34;

In [ ]:
SpatialOperator Stokes = new SpatialOperator(3, 3, QuadOrderFunc.FixedOrder(locOrder), "u", "v", "p", "mom_x", "mom_y", "conti"); 
Stokes.EquationComponents["mom_x"].Add(new Gradient_d(0));
Stokes.EquationComponents["mom_x"].Add(new Viscous(0));
Stokes.EquationComponents["mom_y"].Add(new Gradient_d(1));
Stokes.EquationComponents["mom_y"].Add(new Viscous(1));
Stokes.EquationComponents["conti"].Add(new Divergence());
Stokes.Commit();

In [ ]:
bool locAnsatz            = false;
bool NoBndy               = false;
SpatialOperator LogStokes = new SpatialOperator(3, 3, QuadOrderFunc.FixedOrder(locOrder), "u", "v", "p", "mom_x", "mom_y", "conti");
LogStokes.EquationComponents["mom_x"].Add(new LogGradient_d(0, NoBndy));
LogStokes.EquationComponents["mom_x"].Add(new Viscous(0, locAnsatz, NoBndy));
LogStokes.EquationComponents["mom_y"].Add(new LogGradient_d(1, NoBndy));
LogStokes.EquationComponents["mom_y"].Add(new Viscous(1, locAnsatz, NoBndy));
//LogStokes.EquationComponents["conti"].Add(new Divergence(locAnsatz, NoBndy));
LogStokes.Commit();

In [ ]:
SpatialOperator ConstStokes = new SpatialOperator(3, 3, QuadOrderFunc.Linear(), "u", "v", "p", "mom_x", "mom_y", "conti");
ConstStokes.EquationComponents["mom_x"].Add(new ConstGradient_d(0, NoBndy));
ConstStokes.EquationComponents["mom_x"].Add(new Viscous(0, locAnsatz, NoBndy));
ConstStokes.EquationComponents["mom_y"].Add(new ConstGradient_d(1, NoBndy));
ConstStokes.EquationComponents["mom_y"].Add(new Viscous(1, locAnsatz, NoBndy));
//ConstStokes.EquationComponents["conti"].Add(new Divergence(locAnsatz, NoBndy));
ConstStokes.Commit();

In [ ]:
Viscous.Re               = 1.0;
Viscous.PolynomialDegree = VelB_Corner.Degree;
Viscous.cj               = grdDat_Corner.Cells.cj;

In [ ]:
var StokesMatrix_Corner = new BlockMsrMatrix(varMap_Corner, varMap_Corner);
var StokesAffine_Corner = new double[varMap_Corner.LocalLength];
var mtxBuilder          = Stokes.GetMatrixBuilder(varMap_Corner, null, varMap_Corner);
mtxBuilder.ComputeMatrix(StokesMatrix_Corner, StokesAffine_Corner);

In [ ]:
SinglePhaseField u              = new SinglePhaseField(VelB_Corner,"u");
SinglePhaseField v              = new SinglePhaseField(VelB_Corner,"v");
SinglePhaseField psi            = new SinglePhaseField(PsiB_Corner,"p");
CoordinateVector SolutionCorner = new CoordinateVector(u,v,psi);

In [ ]:
double[] RHS = StokesAffine_Corner.CloneAs();
RHS.ScaleV(-1.0);
//RHS.SaveToTextFile("RHS");
StokesMatrix_Corner.Solve_Direct(SolutionCorner, RHS);

In [ ]:
double thetaGrad = theta * (180.0 / Math.PI);
string pltName = "CornerFlow"+ thetaGrad;
Tecplot("plots/"+pltName+"_singularitySolution", 0.0 , 3, u, v, psi);

Writing output file plots/CornerFlow45_singularitySolution...
done.


In [ ]:
SinglePhaseField uL   = new SinglePhaseField(VelB_Corner,"u");
SinglePhaseField vL   = new SinglePhaseField(VelB_Corner,"v");
SinglePhaseField psiL = new SinglePhaseField(PsiB_Corner,"p");
uL.Acc(1.0, u);
vL.Acc(1.0, v);
psiL.Acc(1.0, psi);
CoordinateVector SolutionLog = new CoordinateVector(uL, vL, psiL);

var LogEval = LogStokes.GetEvaluatorEx(SolutionLog.Mapping.Fields, null, varMap_Corner);
LogEval.Evaluate(1.0, 0.0, SolutionLog);
SolutionLog.SaveToTextFile("SolutionLog");

In [ ]:
SinglePhaseField uC   = new SinglePhaseField(VelB_Corner,"u");
SinglePhaseField vC   = new SinglePhaseField(VelB_Corner,"v");
SinglePhaseField psiC = new SinglePhaseField(PsiB_Corner,"p");
uC.Acc(1.0, u);
vC.Acc(1.0, v);
psiC.Acc(1.0, psi);
CoordinateVector SolutionConst = new CoordinateVector(uC, vC, psiC);

var ConstEval = ConstStokes.GetEvaluatorEx(SolutionConst.Mapping.Fields, null, varMap_Corner);
ConstEval.Evaluate(1.0, 0.0, SolutionConst);
SolutionConst.SaveToTextFile("SolutionConst");

In [ ]:
MultidimensionalArray LSS = MultidimensionalArray.Create(new int[]{RHS.Count(),2});
LSS.SetSubVector(SolutionLog, new int[] {-1, 0});
LSS.SetSubVector(SolutionConst, new int[] {-1, 1});
//LSS.SaveToTextFile("LSS");
//MultidimensionalArray LSST = LSS.Transpose();

In [ ]:
double[] alpha = new double[2];
LSS.LeastSquareSolve(alpha, RHS);

In [ ]:
alpha

index,value
0,-0.007989641808245912
1,0.12374993501359935


In [ ]:
MultidimensionalArray LSSTLSS = MultidimensionalArray.Create(new int[]{2,2});
double[] LSSTRHS = new double[2];
for(int i = 0; i < RHS.Count(); i++){
LSSTLSS[0,0] += SolutionLog[i]*SolutionLog[i];
LSSTLSS[0,1] += SolutionLog[i]*SolutionConst[i];
LSSTLSS[1,0] += SolutionLog[i]*SolutionConst[i];
LSSTLSS[1,1] += SolutionConst[i]*SolutionConst[i];
LSSTRHS[0] += SolutionLog[i]*RHS[i];
LSSTRHS[1] += SolutionConst[i]*RHS[i];
}

In [ ]:
double[] alpha2 = new double[2];
LSSTLSS.Solve(alpha2, LSSTRHS)

In [ ]:
alpha2

index,value
0,-0.00798964180824593
1,0.12374993501359963


In [ ]:
double[] regRHS = RHS.CloneAs();
regRHS.AccV(-alpha[0], SolutionLog);
regRHS.AccV(-alpha[1], SolutionConst);
//regRHS.AccV(-1.0, SolutionLog);
//regRHS.AccV(-1.0, SolutionConst);
//regRHS.SaveToTextFile("regRHS");

In [ ]:
SinglePhaseField uR          = new SinglePhaseField(VelB_Corner,"u");
SinglePhaseField vR          = new SinglePhaseField(VelB_Corner,"v");
SinglePhaseField psiR        = new SinglePhaseField(PsiB_Corner,"p");
CoordinateVector regSolution = new CoordinateVector(uR, vR, psiR);

In [ ]:
StokesMatrix_Corner.Solve_Direct( regSolution, regRHS);

In [ ]:
Tecplot("plots/"+pltName+"_regularizedSolution", 0.0 , 3, uR, vR, psiR);

Writing output file plots/CornerFlow45_regularizedSolution...
done.


In [ ]:
SinglePhaseField psiAnsatz = new SinglePhaseField(PsiB_Corner);
Func<double[], double> logAnsatz = X => alpha[0] * (1.0/lSlip) * Math.Log(Math.Sqrt(X[0].Pow2() + X[1].Pow2())) + alpha[1];
psiAnsatz.ProjectField(logAnsatz);

In [ ]:
SinglePhaseField psi2 = new SinglePhaseField(PsiB_Corner, "p");
psi2.Acc(1.0, psiR);
psi2.Acc(1.0, psiAnsatz);

In [ ]:
Tecplot("plots/"+pltName+"_regainedSolution", 0.0 , 3, psi2);

Writing output file plots/CornerFlow45_regainedSolution...
done.
